# EDA of datasets provided by Kaggle
The purpose of this notebook is to explore each of the datasets provided by Kaggle.
The output will be:
- comments assisting with the general understanding of the data available
- interesting insights that may influence modelling
- recommendations for feature engineering

## Train
Note, this is the largest dataset and the assumption is most of the feature engineering will be built around this dataset.

In [3]:
from config import proj
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.read.parquet(str(proj.Config.paths.get("data_proc").joinpath("train.parquet")))

In [6]:
df.show(5)

+----------+---------+--------+----------+-----------+
|      date|store_nbr|item_nbr|unit_sales|onpromotion|
+----------+---------+--------+----------+-----------+
|2013-04-06|       49|  586969|       7.0|       null|
|2013-04-06|       49|  587069|      21.0|       null|
|2013-04-06|       49|  587156|       6.0|       null|
|2013-04-06|       49|  587186|      22.0|       null|
|2013-04-06|       49|  589403|       9.0|       null|
+----------+---------+--------+----------+-----------+
only showing top 5 rows



Initial ideas
- date range
- any missing dates
- counts of stores, items
- unit sales distribution (overall)
- onpromo distribution over all
- time series plot (any seasonality)
- STL decomposition (trends seasonality)
- check for stationality (mean and variance stationary in time)
- missing promo flags (suggestion of treatment)
- checks for autocorrelation and partial autocorrelation